# Data Processing

This notebook is concentating on getting relational dataframe generated on the previous notebook as an input, performing several processing operations to the data and prepare it for the two main tasks:

1. Performing training with a structured and reliable data
2. Predicting maintenance score as the target variable

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Supporting Functions

In [2]:
def custom_statistics(df):
    """
    Calculate custom statistics for a Pandas DataFrame.
    
    Parameters:
    self (pd.DataFrame): The DataFrame for which to calculate statistics.
    
    Returns:
    pd.DataFrame: A DataFrame containing the calculated statistics.
    """
    # Create an empty DataFrame to hold the statistics
    stats = pd.DataFrame()

    # Calculate mean for each column
    stats['mean'] = df.mean()

    # Calculate median for each column
    stats['median'] = df.median()

    # Calculate mode for each column. Since mode() returns a DataFrame, we take the first mode.
    stats['mode'] = df.mode().iloc[0]

    # Calculate standard deviation for each column
    stats['std'] = df.std()

    # Calculate variance for each column
    stats['var'] = df.var()

    # Calculate minimum value for each column
    stats['min'] = df.min()

    # Calculate 25th percentile (first quartile) for each column
    stats['25%'] = df.quantile(0.25)

    # Calculate 50th percentile (second quartile or median) for each column
    stats['50%'] = df.quantile(0.5)

    # Calculate 75th percentile (third quartile) for each column
    stats['75%'] = df.quantile(0.75)

    # Calculate maximum value for each column
    stats['max'] = df.max()

    # Calculate the count of non-null values for each column
    stats['non-null-proportion'] = df.count() / len(df)

    # Create subplots
    fig, axs = plt.subplots(4, 3, figsize=(15, 20))
    fig.suptitle('Monthly Statistics Overview')

    # Plotting each variable
    stats['mean'].plot(ax=axs[0, 0], kind='line', marker='o', title='Mean')
    stats['median'].plot(ax=axs[0, 1], kind='line', marker='o', title='Median')
    stats['mode'].plot(ax=axs[0, 2], kind='line', marker='o', title='Mode')
    stats['std'].plot(ax=axs[1, 0], kind='line', marker='o', title='Standard Deviation')
    stats['var'].plot(ax=axs[1, 1], kind='line', marker='o', title='Variance')
    stats['min'].plot(ax=axs[1, 2], kind='line', marker='o', title='Minimum')
    stats['25%'].plot(ax=axs[2, 0], kind='line', marker='o', title='25th Percentile')
    stats['50%'].plot(ax=axs[2, 1], kind='line', marker='o', title='50th Percentile')
    stats['75%'].plot(ax=axs[2, 2], kind='line', marker='o', title='75th Percentile')
    stats['max'].plot(ax=axs[3, 0], kind='line', marker='o', title='Maximum')
    stats['non-null-proportion'].plot(ax=axs[3, 1], kind='line', marker='o', title='Non-null Proportion')

    # Hide the last empty subplot
    axs[3, 2].axis('off')

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    return stats

# Function to fill the new dataframe with commit counts
def fill_counts(row, row_index, df, information):
    for entry in row:
        year_month = f"{entry['month']:02d}-{entry['year']}"
        if year_month in df.columns:
            df.at[row_index, year_month] = entry[information]


def implement_months(repository):
    if 'date_month' in repository.columns:
        # Convert 'date_month' column to datetime format
        repository['date_month'] = pd.to_datetime(repository['date_month'])

        # Extract year and month from the 'date_month' column
        repository['year'] = repository['date_month'].dt.year
        repository['month'] = repository['date_month'].dt.month

        repository.drop(columns=['date_month'], inplace=True)
    
    repository = repository.sort_values(by=['year', 'month'], ascending=True)
    repository.reset_index(inplace=True, drop=True)
    repository['month'] = repository['month'].astype(str).str.zfill(2)
    repository['date'] = repository['month'].astype(str) + '-' + repository['year'].astype(str)

    if repository.empty:
        return None

    # Create a complete date range from the minimum to maximum dates in the original data
    # max and min values likely to be a fixed value for all repositories
    min_year, min_month = repository['year'].iloc[0], repository['month'].iloc[0]
    max_year, max_month = repository['year'].iloc[-1], repository['month'].iloc[-1]
    min_date = f"{min_month}-{min_year}"
    max_date = f"{max_month}-{max_year}"
    date_range = pd.date_range(start=min_date, end=max_date, freq='MS')

    # Create a DataFrame from the date range
    date_df = pd.DataFrame({'date': date_range})

    # Extract year and month from the date range
    date_df['year'] = date_df['date'].dt.year
    date_df['month'] = date_df['date'].dt.month.astype(str).str.zfill(2)

    # Convert the date column to the same format as in your original DataFrame
    date_df['date'] = date_df['date'].dt.strftime('%m-%Y')

    # Merge the original DataFrame with the date DataFrame to fill in missing values
    repository = pd.merge(date_df, repository, on=['year', 'month', 'date'], how='left')

    repository = repository.fillna(0)

    return repository

def array_to_duration(repository, column):
    repository["duration"] = repository[column].apply(lambda x: x[0] * 30 + x[1] + x[2] / (24 * 3600) + x[3] / (24 * 3600 * 10 ** 9) if x is not None and len(x) == 4 else 9999.9999)
    repository["duration"].replace(np.inf, repository["duration"].median(), inplace=True)
    repository.drop(columns=[column], inplace=True)
    return repository

def extract_comments_and_issues(json_data):
    issue_df = json_data[['issue.createdAt', 'issue.creatorRole', 'comments']].copy()
    issue_df['issue.createdAt'] = issue_df['issue.createdAt'].apply(lambda x: pd.to_datetime(x))
    issue_df['month'] = issue_df['issue.createdAt'].dt.month
    issue_df['year'] = issue_df['issue.createdAt'].dt.year
    issue_df['date'] = issue_df['month'].astype(str).str.zfill(2) + '-' + issue_df['year'].astype(str)
    issue_df = issue_df.rename(columns={'issue.creatorRole': 'creatorRole'})
    issue_df = issue_df.drop(columns=['issue.createdAt'])
    issue_df

    comments_list = []
    for comments in issue_df['comments']:
        comments_list.extend(comments)
    issue_df = issue_df.drop(columns=['comments'])

    if comments_list != []:    
        comments_df = pd.json_normalize(comments_list)
        comments_df['createdAt'] = comments_df['createdAt'].apply(lambda x: pd.to_datetime(x))
        comments_df['month'] = comments_df['createdAt'].dt.month
        comments_df['year'] = comments_df['createdAt'].dt.year
        comments_df = comments_df.drop(columns=['createdAt', 'creator'])

        issue_df = pd.concat([issue_df, comments_df]).reset_index(drop=True)

    # Filtering valid roles
    valid_roles = ['COLLABORATOR', 'MEMBER', 'OWNER']
    issue_df = issue_df[issue_df['creatorRole'].isin(valid_roles)]

    grouped_counts = issue_df.groupby(['month', 'year']).size().reset_index(name='sum')
    # grouped_counts['month'] = grouped_counts['month'].astype('Int64')
    # grouped_counts['year'] = grouped_counts['year'].astype('Int64')
    grouped_counts['sum'] = grouped_counts['sum'].astype('Int64')
    grouped_counts = implement_months(grouped_counts)
    return grouped_counts

## Loading the Dataset

In [3]:
df = pd.read_parquet('Data/Processed/procesed_relational_dataset.parquet')
df

,project_name,github_link,project_url,project_id,metric_results,get_commits_per_month,get_avg_issue_close_time_per_month,get_avg_pull_request_close_time_per_month,get_new_issue_author_count_per_month,get_new_pull_request_author_count_per_month,get_avg_issue_response_time_per_month,get_avg_pull_request_merge_time_per_month,get_closed_issues_per_month,get_closed_pull_requests_per_month,get_commits_count_by_author_descending,get_label_issue_and_pull_request_count,get_issue_author_comment_count,get_discussion_author_comment_count,get_project_information,get_issues_and_issue_comments
0,netto,https://github.com/0-k/netto,0-k%2Fnetto,R_kgDOIU4I5A,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 47, 'month': 11, 'year': 2022}, ...",None,None,None,None,None,None,None,None,"[{'author_login': '0-k', 'commit_count': 65}]","[{'issue_label_count': 0, 'l.name': 'help want...",None,None,"[{'archivedAt': '0001-01-01T01:01:01+00:00', '...",None
1,GameGui,https://github.com/00001h/gamegui,00001h%2Fgamegui,R_kgDOGTAsgw,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 18, 'month': 1, 'year': 2022}, {...",None,None,None,None,None,None,None,None,"[{'author_login': '00001H', 'commit_count': 53}]","[{'issue_label_count': 0, 'l.name': 'Focus on'...",None,None,"[{'archivedAt': '2023-03-28T13:54:56+00:00', '...",None
2,apiutils,https://github.com/007gzs/apiutils,007gzs%2Fapiutils,MDEwOlJlcG9zaXRvcnkyMzI3MDY3OTU=,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 5, 'month': 8, 'year': 2020}, {'...",None,None,None,None,None,None,None,None,"[{'author_login': '007gzs', 'commit_count': 35...","[{'issue_label_count': 0, 'l.name': 'invalid',...",None,None,"[{'archivedAt': '0001-01-01T01:01:01+00:00', '...",None
3,avatars,https://github.com/007gzs/avatars,007gzs%2Favatars,MDEwOlJlcG9zaXRvcnkyMjU3NTA3Mjc=,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 11, 'month': 12, 'year': 2019}]",None,None,None,None,None,None,None,None,"[{'author_login': '007gzs', 'commit_count': 11}]","[{'issue_label_count': 0, 'l.name': 'bug', 'pu...",None,None,"[{'archivedAt': '0001-01-01T01:01:01+00:00', '...",None
4,baijiayun,https://github.com/007gzs/baijiayun,007gzs%2Fbaijiayun,MDEwOlJlcG9zaXRvcnkyNjQwODM1NTQ=,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 7, 'month': 5, 'year': 2020}]",None,None,None,None,None,None,None,None,"[{'author_login': 'default', 'commit_count': 6...","[{'issue_label_count': 0, 'l.name': 'enhanceme...",None,None,"[{'archivedAt': '0001-01-01T01:01:01+00:00', '...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30499,aiohttp_traversal,https://github.com/zzzsochi/aiohttp_traversal,zzzsochi%2Faiohttp_traversal,MDEwOlJlcG9zaXRvcnkzNjYxMjMyNQ==,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 2, 'month': 3, 'year': 2017}, {'...",None,"[{'AVG(open_duration)': [0, 1, 2995, 333333333...","[{'date_month': '2016-05-01T00:00:00+00:00', '...","[{'date_month': '2017-03-01T00:00:00+00:00', '...","[{'avg_response_time': [0, 0, 4897, 0], 'month...","[{'avg_merge_duration': [0, 0, 5428, 0], 'mont...","[{'closed_issues': 0, 'date_month': '2016-05-0...","[{'closed_pull_requests': 3, 'date_month': '20...","[{'author_login': 'zzzsochi', 'commit_count': ...","[{'issue_label_count': 0, 'l.name': 'duplicate...","[{'comment_count': 1, 'u.login': 'kxepal'}]",None,"[{'archivedAt': '0001-01-01T01:01:01+00:00', '...",[{'comments': [{'createdAt': '2016-05-02T19:56...
30500,cats,https://github.com/zzzsochi/cats,zzzsochi%2Fcats,MDEwOlJlcG9zaXRvcnk0OTA5NzIxMA==,"{'get_avg_issue_close_time_per_month': [], 'ge...","[{'COUNT(c)': 6, 'month': 1, 'year': 2016}]",None,None,"[{'date_month': '2016-03-01T00:00:00+00:00', '...",None,"[{'avg_response_time': [0, 0, 20118, 0], 'mont...",None,"[{'closed_issues': 0, 'date_month': '2016-03-0...",None,"[{'author_login': 'zzzsochi', 'commit_count': 6}]","[{'issue_label_count': 0, 'l.name': 'bug', 'pu...","[{'co

## Define the data interval

In [4]:
# Define the start and end dates (we are getting three months before of the starting date, because each month should consider the activities based on the last 90 days)
start_year, start_month = 2023, 1
end_year, end_month = 2024, 7

# Generate the list of months between start and end dates
months = pd.date_range(start=f"{start_month}-{start_year}", end=f"{end_month}-{end_year}", freq='MS').strftime("%m-%Y").tolist()

## Get Commits per Month

In [5]:
commit_per_month = df['get_commits_per_month']

# Create a new dataframe with months as columns
commit_per_month_structured = pd.DataFrame(index=commit_per_month.index, columns=months)

# Apply the function to each row
for i in range(len(commit_per_month)):
    fill_counts(commit_per_month.iloc[i], i, commit_per_month_structured, 'COUNT(c)')

commit_per_month_structured.fillna(0, inplace=True)

# commit_per_month_structured.fillna(0, inplace=True)
commit_per_month_structured

/var/folders/6f/nfwqn6h53sx5b2b3hqq_19r80000gn/T/ipykernel_33830/203857505.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  commit_per_month_structured.fillna(0, inplace=True)


,01-2023,02-2023,03-2023,04-2023,05-2023,06-2023,07-2023,08-2023,09-2023,10-2023,11-2023,12-2023,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30499,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30502,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Statistics

In [222]:
# commit_per_month_structured_stats = custom_statistics(commit_per_month_structured)
# commit_per_month_structured_stats

### Saving the data

In [223]:
commit_per_month_structured.fillna(0, inplace=True)
commit_per_month_structured.to_parquet('Data/Processed/Metrics/commit_per_month.parquet')

## Average Issue Close Time per Month

In [224]:
# avg_issue_close_time_per_month = df['get_avg_issue_close_time_per_month']

# # Create a new dataframe with months as columns
# avg_issue_close_time_per_month_structured_duration = pd.DataFrame(index=avg_issue_close_time_per_month.index, columns=months)
# avg_issue_close_time_per_month_structured_count = pd.DataFrame(index=avg_issue_close_time_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(avg_issue_close_time_per_month)):
#     if avg_issue_close_time_per_month.iloc[i] is None:
#         continue
#     x = pd.json_normalize(avg_issue_close_time_per_month.iloc[i])
#     df_entry = array_to_duration(x, 'AVG(open_duration)')
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in avg_issue_close_time_per_month_structured_duration.columns:
#             avg_issue_close_time_per_month_structured_duration.at[i, j] = df_entry[df_entry['date'] == j]['duration'].values[0]
#             avg_issue_close_time_per_month_structured_count.at[i, j] = df_entry[df_entry['date'] == j]['COUNT(open_duration)'].values[0]

# avg_issue_close_time_per_month_structured_duration.replace(0, np.nan, inplace=True)
# avg_issue_close_time_per_month_structured_count.replace(0, np.nan, inplace=True)
# avg_issue_close_time_per_month_structured_count

### Statistics

In [225]:
# avg_issue_close_time_per_month_structured_duration_stats = custom_statistics(avg_issue_close_time_per_month_structured_duration)
# avg_issue_close_time_per_month_structured_duration_stats

### Statistics

In [226]:
# avg_issue_close_time_per_month_structured_count_stats = custom_statistics(avg_issue_close_time_per_month_structured_count)
# avg_issue_close_time_per_month_structured_count_stats

### Saving the data

In [227]:
# avg_issue_close_ti
# avg_issue_close_time_per_month_structured_count.to_parquet('Data/Processed/Metrics/avg_issue_close_time_per_month_count.parquet')me_per_month_structured_duration.fillna(0, inplace=True)
# avg_issue_close_time_per_month_structured_count.fillna(0, inplace=True)
# avg_issue_close_time_per_month_structured_duration.to_parquet('Data/Processed/Metrics/avg_issue_close_time_per_month_duration.parquet')

## Average PR Close Time Per Month

In [228]:
# avg_pull_request_close_time_per_month = df['get_avg_pull_request_close_time_per_month']
# # Create a new dataframe with months as columns
# avg_pull_request_close_time_per_month_structured = pd.DataFrame(index=avg_pull_request_close_time_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(avg_pull_request_close_time_per_month)):
#     if avg_pull_request_close_time_per_month.iloc[i] is None:
#         continue
#     x = pd.json_normalize(avg_pull_request_close_time_per_month.iloc[i])
#     df_entry = array_to_duration(x, 'AVG(open_duration)')
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in avg_pull_request_close_time_per_month_structured.columns:
#             avg_pull_request_close_time_per_month_structured.at[i, j] = df_entry[df_entry['date'] == j]['duration'].values[0]
            

# avg_pull_request_close_time_per_month_structured.replace(0, np.nan, inplace=True)
# avg_pull_request_close_time_per_month_structured

### Statistics

In [229]:
# avg_pull_request_close_time_per_month_structured_stats = custom_statistics(avg_pull_request_close_time_per_month_structured)
# avg_pull_request_close_time_per_month_structured_stats

### Saving the data

In [230]:
# avg_pull_request_close_time_per_month_structured.fillna(0, inplace=True)
# avg_pull_request_close_time_per_month_structured.to_parquet('Data/Processed/Metrics/avg_pull_request_close_time_per_month.parquet')

## New Issue Author Count per Month

In [231]:
# new_issue_author_count_per_month = df['get_new_issue_author_count_per_month']

# # Create a new dataframe with months as columns
# new_issue_author_count_per_month_structured = pd.DataFrame(index=new_issue_author_count_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(new_issue_author_count_per_month)):
#     if new_issue_author_count_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(new_issue_author_count_per_month.iloc[i])
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in new_issue_author_count_per_month_structured.columns:
#             new_issue_author_count_per_month_structured.at[i, j] = df_entry[df_entry['date'] == j]['new_authors_count'].values[0]

# new_issue_author_count_per_month_structured

### Statistics

In [232]:
# new_issue_author_count_per_month_structured_stats = custom_statistics(new_issue_author_count_per_month_structured)
# new_issue_author_count_per_month_structured_stats

### Saving the data

In [233]:
# new_issue_author_count_per_month_structured.fillna(0, inplace=True)
# new_issue_author_count_per_month_structured.to_parquet('Data/Processed/Metrics/new_issue_author_count_per_month.parquet')

## New PR Author Count per Month

In [234]:
# new_pull_request_author_count_per_month = df['get_new_pull_request_author_count_per_month']

# # Create a new dataframe with months as columns
# new_pull_request_author_count_per_month_structured = pd.DataFrame(index=new_pull_request_author_count_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(new_pull_request_author_count_per_month)):
#     if new_pull_request_author_count_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(new_pull_request_author_count_per_month.iloc[i])
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in new_pull_request_author_count_per_month_structured.columns:
#             new_pull_request_author_count_per_month_structured.at[i, j] = df_entry[df_entry['date'] == j]['new_authors_count'].values[0]

# new_pull_request_author_count_per_month_structured

### Statistics

In [235]:
# new_pull_request_author_count_per_month_structured_stats = custom_statistics(new_pull_request_author_count_per_month_structured)
# new_pull_request_author_count_per_month_structured_stats

### Saving the data

In [236]:
# new_pull_request_author_count_per_month_structured.fillna(0, inplace=True)
# new_pull_request_author_count_per_month_structured.to_parquet('Data/Processed/Metrics/new_pull_request_author_count_per_month.parquet')

## Average Issue Response Time per Month

In [237]:
# avg_issue_response_time_per_month = df['get_avg_issue_response_time_per_month']

# # Create a new dataframe with months as columns
# avg_issue_response_time_per_month_structured = pd.DataFrame(index=avg_issue_response_time_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(avg_issue_response_time_per_month)):
#     if avg_issue_response_time_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(avg_issue_response_time_per_month.iloc[i])
#     df_entry = array_to_duration(df_entry, 'avg_response_time')
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in avg_issue_response_time_per_month_structured.columns:
#             avg_issue_response_time_per_month_structured.at[i, j] = df_entry[df_entry['date'] == j]['duration'].values[0]

# avg_issue_response_time_per_month_structured

### Statistics

In [238]:
# avg_issue_response_time_per_month_structured_stats = custom_statistics(avg_issue_response_time_per_month_structured)
# avg_issue_response_time_per_month_structured_stats

### Saving the data

In [239]:
# avg_issue_response_time_per_month_structured.fillna(0, inplace=True)
# avg_issue_response_time_per_month_structured.to_parquet('Data/Processed/Metrics/avg_issue_response_time_per_month.parquet')

## Average PR Merge Time per Month

In [240]:
# avg_pull_request_merge_time_per_month = df['get_avg_pull_request_merge_time_per_month']

# # Create a new dataframe with months as columns
# avg_pull_request_merge_time_per_month_structured = pd.DataFrame(index=avg_pull_request_merge_time_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(avg_pull_request_merge_time_per_month)):
#     if avg_pull_request_merge_time_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(avg_pull_request_merge_time_per_month.iloc[i])
#     df_entry = array_to_duration(df_entry, 'avg_merge_duration')
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in avg_pull_request_merge_time_per_month_structured.columns:
#             avg_pull_request_merge_time_per_month_structured.at[i, j] = df_entry[df_entry['date'] == j]['duration'].values[0]

# avg_pull_request_merge_time_per_month_structured

### Statistics

In [241]:
# avg_pull_request_close_time_per_month_structured_stats = custom_statistics(avg_pull_request_close_time_per_month_structured)
# avg_pull_request_close_time_per_month_structured_stats

### Saving the data

In [242]:
# avg_pull_request_merge_time_per_month_structured.fillna(0, inplace=True)
# avg_pull_request_merge_time_per_month_structured.to_parquet('Data/Processed/Metrics/avg_pull_request_merge_time_per_month.parquet')

## Closed Issues per Month

In [243]:
# closed_issues_per_month = df['get_closed_issues_per_month']

# # Create a new dataframe with months as columns
# closed_issues_per_month_opened_issues_structured = pd.DataFrame(index=closed_issues_per_month.index, columns=months)
# closed_issues_per_month_closed_issues_structured = pd.DataFrame(index=closed_issues_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(closed_issues_per_month)):
#     if closed_issues_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(closed_issues_per_month.iloc[i])
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in closed_issues_per_month_opened_issues_structured.columns:
#             closed_issues_per_month_opened_issues_structured.at[i, j] = df_entry[df_entry['date'] == j]['opened_issues'].values[0]
#             closed_issues_per_month_closed_issues_structured.at[i, j] = df_entry[df_entry['date'] == j]['closed_issues'].values[0]

# closed_issues_per_month_closed_issues_structured

### Statistics

In [244]:
# closed_issues_per_month_opened_issues_structured_stats = custom_statistics(closed_issues_per_month_opened_issues_structured)
# closed_issues_per_month_opened_issues_structured_stats

In [245]:
# closed_issues_per_month_closed_issues_structured_stats = custom_statistics(closed_issues_per_month_closed_issues_structured)
# closed_issues_per_month_closed_issues_structured_stats

### Saving the data

In [246]:
# closed_issues_per_month_closed_issues_structured.fillna(0, inplace=True)
# closed_issues_per_month_closed_issues_structured.to_parquet('Data/Processed/Metrics/closed_issues_per_month_closed_issues.parquet')
# closed_issues_per_month_closed_issues_structured.fillna(0, inplace=True)
# closed_issues_per_month_closed_issues_structured.to_parquet('Data/Processed/Metrics/closed_issues_per_month_closed_issues.parquet')

## Closed PR per Month

In [247]:
# closed_pull_requests_per_month = df['get_closed_pull_requests_per_month']

# # Create a new dataframe with months as columns
# closed_pull_requests_per_month_open_pull_requests_structured = pd.DataFrame(index=closed_pull_requests_per_month.index, columns=months)
# closed_pull_requests_per_month_closed_pull_requests_structured = pd.DataFrame(index=closed_pull_requests_per_month.index, columns=months)

# # Apply the function to each row
# for i in range(len(closed_pull_requests_per_month)):
#     if closed_pull_requests_per_month.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(closed_pull_requests_per_month.iloc[i])
#     df_entry = implement_months(df_entry)
#     for j in df_entry['date']:
#         if j in closed_pull_requests_per_month_open_pull_requests_structured.columns:
#             closed_pull_requests_per_month_open_pull_requests_structured.at[i, j] = df_entry[df_entry['date'] == j]['open_pull_requests'].values[0]
#             closed_pull_requests_per_month_closed_pull_requests_structured.at[i, j] = df_entry[df_entry['date'] == j]['closed_pull_requests'].values[0]

# closed_pull_requests_per_month_closed_pull_requests_structured

### Statistics

In [248]:
# closed_pull_requests_per_month_open_pull_requests_structured_stats = custom_statistics(closed_pull_requests_per_month_open_pull_requests_structured)
# closed_pull_requests_per_month_open_pull_requests_structured_stats

In [249]:
# closed_pull_requests_per_month_closed_pull_requests_structured_stats = custom_statistics(closed_pull_requests_per_month_closed_pull_requests_structured)
# closed_pull_requests_per_month_closed_pull_requests_structured_stats

### Saving the data

In [250]:
# closed_pull_requests_per_month_open_pull_requests_structured.fillna(0, inplace=True)
# closed_pull_requests_per_month_open_pull_requests_structured.to_parquet('Data/Processed/Metrics/closed_pull_requests_per_month_open_pull_requests.parquet')
# closed_pull_requests_per_month_closed_pull_requests_structured.fillna(0, inplace=True)
# closed_pull_requests_per_month_closed_pull_requests_structured.to_parquet('Data/Processed/Metrics/closed_pull_requests_per_month_closed_pull_requests.parquet')

## Commits Count by Author

In [251]:
# # Assuming df is already defined
# commits_count_by_author_descending = df['get_commits_count_by_author_descending']

# # Calculate the maximum number of authors in any row
# max_length = max(commits_count_by_author_descending.apply(lambda x: len(x) if x is not None else 0))

# # Prepare columns for the maximum number of authors
# columns = [f"Author_{i+1}" for i in range(max_length)]

# # Initialize a dictionary to collect the data
# data_dict = {col: [None] * len(commits_count_by_author_descending) for col in columns}

# # Populate the dictionary with commit counts
# for i in range(len(commits_count_by_author_descending)):
#     if commits_count_by_author_descending.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(commits_count_by_author_descending.iloc[i])
#     length = len(df_entry['commit_count'])
#     for j in range(length):
#         data_dict[f"Author_{j+1}"][i] = int(df_entry.at[j, 'commit_count'])

# # Convert the dictionary to a DataFrame
# commits_count_by_author_descending_structured = pd.DataFrame(data_dict, index=commits_count_by_author_descending.index)

# commits_count_by_author_descending_structured


### Statistics

In [252]:
# commits_count_by_author_descending_structured_stats = custom_statistics(commits_count_by_author_descending_structured)
# commits_count_by_author_descending_structured_stats

### Saving the data

In [253]:
# commits_count_by_author_descending_structured.fillna(0, inplace=True)
# commits_count_by_author_descending_structured.to_parquet('Data/Processed/Metrics/commits_count_by_author_descending.parquet')

## Issue Author Comment Count

In [254]:
# issue_author_comment_count = df['get_issue_author_comment_count']

# # Calculate the maximum number of authors in any row
# max_length = max(issue_author_comment_count.apply(lambda x: len(x) if x is not None else 0))

# # Prepare columns for the maximum number of authors
# columns = [f"Author_{i+1}" for i in range(max_length)]

# # Initialize a dictionary to collect the data
# data_dict = {col: [None] * len(issue_author_comment_count) for col in columns}

# # Populate the dictionary with commit counts
# for i in range(len(issue_author_comment_count)):
#     if issue_author_comment_count.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(issue_author_comment_count.iloc[i])
#     length = len(df_entry['comment_count'])
#     for j in range(length):
#         data_dict[f"Author_{j+1}"][i] = int(df_entry.at[j, 'comment_count'])

# # Convert the dictionary to a DataFrame
# issue_author_comment_count_structured = pd.DataFrame(data_dict, index=issue_author_comment_count.index)

# issue_author_comment_count_structured

### Statistics

In [255]:
# issue_author_comment_count_structured_stats = custom_statistics(issue_author_comment_count_structured)
# issue_author_comment_count_structured_stats

### Saving the data

In [256]:
# issue_author_comment_count_structured.fillna(0, inplace=True)
# issue_author_comment_count_structured.to_parquet('Data/Processed/Metrics/issue_author_comment_count.parquet')

## Discussion Author Comment Count

I personally didn't find any efficient way to utilize this metric. Also, most of the data is NaN.

In [257]:
# discussion_author_comment_count = df['get_discussion_author_comment_count']

# # Calculate the maximum number of authors in any row
# max_length = max(discussion_author_comment_count.apply(lambda x: len(x) if x is not None else 0))

# # Prepare columns for the maximum number of authors
# columns = [f"Author_{i+1}" for i in range(max_length)]

# # Initialize a dictionary to collect the data
# data_dict = {col: [None] * len(discussion_author_comment_count) for col in columns}

# # Populate the dictionary with commit counts
# for i in range(len(discussion_author_comment_count)):
#     if discussion_author_comment_count.iloc[i] is None:
#         continue
#     df_entry = pd.json_normalize(discussion_author_comment_count.iloc[i])
#     length = len(df_entry['comment_count'])
#     for j in range(length):
#         data_dict[f"Author_{j+1}"][i] = int(df_entry.at[j, 'comment_count'])

# # Convert the dictionary to a DataFrame
# discussion_author_comment_count_structured = pd.DataFrame(data_dict, index=discussion_author_comment_count.index)

# discussion_author_comment_count_structured

In [258]:
# discussion_author_comment_count_structured_stats = custom_statistics(discussion_author_comment_count_structured)
# discussion_author_comment_count_structured_stats

In [259]:
# discussion_author_comment_count_structured.fillna(0, inplace=True)
# discussion_author_comment_count_structured.to_parquet('Data/Processed/Metrics/discussion_author_comment_count.parquet')

## Get Project Information

In [260]:
project_information = pd.json_normalize(df['get_project_information'].apply(lambda x: x[0] if x is not None else None))
# Convert "archivedAt" and "createdAt" columns to datetime type
project_information["archivedAt"] = project_information["archivedAt"].apply(lambda x: pd.to_datetime(x) if x != "0001-01-01T01:01:01+00:00" else pd.to_datetime("1970-01-01T00:00:00+00:00"))
project_information["createdAt"] = pd.to_datetime(project_information["createdAt"])

# Extract year and month
project_information["create_year"] = project_information["createdAt"].dt.year.astype('Int64')
project_information["create_month"] = project_information["createdAt"].dt.month.astype('Int64')
project_information["archive_year"] = project_information["archivedAt"].dt.year.astype('Int64')
project_information["archive_month"] = project_information["archivedAt"].dt.month.astype('Int64')
isArchived = project_information["isArchived"].astype('Int64')
project_information.drop(columns=['archivedAt', 'createdAt'], inplace=True)

# project_information = project_information.astype(int)
project_information

,isArchived,create_year,create_month,archive_year,archive_month
0,False,2018,1,1970,1
1,False,2016,2,1970,1
2,False,2020,6,1970,1
3,False,2019,7,1970,1
4,False,2020,2,1970,1
...,...,...,...,...,...
105,False,2012,12,1970,1
106,False,2021,5,1970,1
107,False,2012,2,1970,1
108,False,2011,5,1970,1


In [263]:
# Create a new dataframe with months as columns
project_information_structured = pd.DataFrame(index=project_information.index, columns=months)

# Function to populate the dataframe
def populate_structured_df(row):
    create_col = f"{row['create_month']:02d}-{row['create_year']}"
    archive_col = f"{row['archive_month']:02d}-{row['archive_year']}"

    print("create_col = {}\t, archive_col = {}".format(create_col, archive_col))
     
    # if row['create_year'] <= start_year and row['create_month'] < start_month:



    # start_year, start_month = 2023, 1
    # end_year, end_month = 2024, 7


# Apply the function to each row in the original dataframe
project_information.apply(populate_structured_df, axis=1)
# project_information_structured = project_information_structured.fillna(False)
project_information_structured

create_col = 01-2018	, archive_col = 01-1970
create_col = 02-2016	, archive_col = 01-1970
create_col = 06-2020	, archive_col = 01-1970
create_col = 07-2019	, archive_col = 01-1970
create_col = 02-2020	, archive_col = 01-1970
create_col = 07-2017	, archive_col = 01-1970
create_col = 08-2020	, archive_col = 01-1970
create_col = 02-2021	, archive_col = 01-1970
create_col = 08-2014	, archive_col = 01-1970
create_col = 11-2021	, archive_col = 01-1970
create_col = 04-2020	, archive_col = 01-1970
create_col = 01-2021	, archive_col = 01-1970
create_col = 11-2017	, archive_col = 01-1970
create_col = 05-2021	, archive_col = 01-1970
create_col = 08-2018	, archive_col = 01-1970
create_col = 11-2020	, archive_col = 01-1970
create_col = 05-2016	, archive_col = 01-1970
create_col = 05-2020	, archive_col = 01-1970
create_col = 07-2021	, archive_col = 01-1970
create_col = 01-2021	, archive_col = 01-1970
create_col = 02-2020	, archive_col = 01-1970
create_col = 03-2013	, archive_col = 01-1970
create_col

,01-2023,02-2023,03-2023,04-2023,05-2023,06-2023,07-2023,08-2023,09-2023,10-2023,11-2023,12-2023,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# # Identify the first and last columns
# first_col = project_information_structured.columns[0]
# last_col = project_information_structured.columns[-1]

# # Filter the dataframe based on the values of the first and last columns
# filtered_df = project_information_structured[(project_information_structured[first_col] == False) & (project_information_structured[last_col] == False)]

# # Further filter to include only rows where at least one column in between is True
# columns_in_between = project_information_structured.columns[1:-1]
# filtered_df = filtered_df[filtered_df[columns_in_between].any(axis=1)]

# filtered_df

In [46]:
# Save to Parquet
project_information_structured.to_parquet('Data/Processed/Metrics/project_information.parquet')

In [211]:
issues = df["get_issues_and_issue_comments"]

# Create a new dataframe with months as columns
issues_structured = pd.DataFrame(index=issues.index, columns=months)

# # Apply the function to each row
for i in range(len(issues)):
    if issues[i] is None:
        continue
    inp = pd.json_normalize(issues[i])
    df_entry = extract_comments_and_issues(inp)
    if df_entry is None:
        continue
    for j in df_entry['date']:
        if j in issues_structured.columns:
            issues_structured.at[i, j] = df_entry[df_entry['date'] == j]['sum'].values[0]

issues_structured.fillna(0, inplace=True)
issues_structured

,01-2024,02-2024,03-2024,04-2024,05-2024,06-2024,07-2024
0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,5,1,0
4,2,4,7,3,4,1,2
...,...,...,...,...,...,...,...
105,0,0,0,0,2,0,0
106,0,0,0,0,0,0,0
107,0,1,0,0,0,0,0
108,9,3,0,0,0,1,0


In [212]:
# Save to Parquet
issues_structured.to_parquet('Data/Processed/Metrics/issues.parquet')

In [ ]:
# TODO: Remember about Neo4J GraphQL Databse and metric schema